In [22]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
#clear_output()

In [3]:
!pip install -U lightrag[ollama]

In [16]:
!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display

In [5]:
!pip install faiss-cpu


import re
import unicodedata
import pandas as pd

# Cargar datos del CSV
df = pd.read_csv('basecsvf.csv')

df['Respuesta'] = df['Respuesta'].apply(lambda x: str(x) if pd.notnull(x) else '')
df['Pregunta'] = df['Pregunta'].apply(lambda x: str(x) if pd.notnull(x) else '')

def normalize(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Pregunta'] = df['Pregunta']
df['Respuesta'] = df['Respuesta']

# Concatenar preguntas y respuestas
df['PreguntaRespuesta'] = df['Pregunta'] + " [SEP] " + df['Respuesta']

# Obtener la lista de textos concatenados
preguntas_respuestas = df['PreguntaRespuesta'].tolist()

from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'sentence-transformers/all-mpnet-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

embeddings = get_embeddings(preguntas_respuestas)

import faiss
import numpy as np

d = 768

vectors = np.array([embedding.numpy() for embedding in embeddings], dtype='float32')

index = faiss.IndexFlatL2(d)

index.add(vectors)

faiss.write_index(index, 'mi_indice_faiss.index')

index = faiss.read_index('mi_indice_faiss.index')

In [23]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = 'sentence-transformers/all-mpnet-base-v2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


nueva_pregunta = "Quién es el director de carrera?"

nueva_embedding = get_embeddings([nueva_pregunta])[0].numpy().astype('float32')

distances, indices = index.search(np.array([nueva_embedding]), k=3)

similarity_percentage = 100 * (1 - distances[0][0])
print(f"Similitud: {similarity_percentage}%")

max_index = len(preguntas_respuestas) - 1
valid_indices = [i for i in indices[0] if i <= max_index]
respuestas_cercanas = [preguntas_respuestas[i] for i in valid_indices]

base_datos_vectorial = "\n".join(respuestas_cercanas)

print(base_datos_vectorial)
base_datos_vectorial = respuestas_cercanas
respuestas_cercanas

Similitud: -644.6331977844238%
¿Quién es el Director de la Carrera de Ingenieria Mecatronica SEDE Latacunga? [SEP] El Director vigente de la carrera de Ing Mecatronica SEDE Latacunga es Darío Mendoza C., Máster. Si necesitas contactar con el Director de Ing Mecatronica, este es su correo institucional: djmendoza@espe.edu.ec
¿Quién es el Director de la Carrera de Ingenieria Mecanica? [SEP] El Director vigente de la carrera de Ing Mecanica es el Ing. Jaime F. Echeverría Y. Si necesitas contactar con el Director de Ing Mecanica, este es su correo institucional: jfecheverria@espe.edu.ec
¿Quién es el Director de la Carrera de Ingenieria en Tecnologias de la Informacion? [SEP] El Director vigente de la carrera de Ing en Tecnologias de la Informacion es el Ing. Fidel Castro, MSc. Si necesitas contactar con el Director de Ing en Tecnologias de la Informacion, este es su correo institucional: flcastro@espe.edu.ec


['¿Quién es el Director de la Carrera de Ingenieria Mecatronica SEDE Latacunga? [SEP] El Director vigente de la carrera de Ing Mecatronica SEDE Latacunga es Darío Mendoza C., Máster. Si necesitas contactar con el Director de Ing Mecatronica, este es su correo institucional: djmendoza@espe.edu.ec',
 '¿Quién es el Director de la Carrera de Ingenieria Mecanica? [SEP] El Director vigente de la carrera de Ing Mecanica es el Ing. Jaime F. Echeverría Y. Si necesitas contactar con el Director de Ing Mecanica, este es su correo institucional: jfecheverria@espe.edu.ec',
 '¿Quién es el Director de la Carrera de Ingenieria en Tecnologias de la Informacion? [SEP] El Director vigente de la carrera de Ing en Tecnologias de la Informacion es el Ing. Fidel Castro, MSc. Si necesitas contactar con el Director de Ing en Tecnologias de la Informacion, este es su correo institucional: flcastro@espe.edu.ec']

In [24]:
# Paso 5: Voy a Generar respuesta con el LLM
qa_template = r"""<SYS>
Eres una IA llamada ESPE Chatbot, respuestas SOLAMENTE relacionadas a la Universidad de las Fuerzas Armadas ESPE.
El usuario te va a hacer preguntas y tienes que responder en base a la siguiente base_datos_vectorial solo si está relacionado con la pregunta del user.
Si hay nombres, teléfonos, direciones o información relacionada en la base_datos_vectorial, da es información porque es segura.
La respuesta que otorgues debe ser natural, amigable y precisa.
Si no encuentras una respuesta adecuada en la base_datos_vectorial, proporciona una respuesta neutral sin mencionar base_datos_vectorial o pregunta de vuelta.
base_datos_vectorial (Escoge solo la respuesta más cercana a la pregunta):{{base_datos_vectorial}}
User: {{input_str}}
You:
<SYS>"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input), "base_datos_vectorial": base_datos_vectorial})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input), "base_datos_vectorial": base_datos_vectorial})

model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)
output = qa.call({"input_str": nueva_pregunta})
display(Markdown(f"**Answer:** {output}"))

**Answer:** GeneratorOutput(data='Buenos días! Puedo ayudarte con eso. La respuesta a tu pregunta sería que dependiendo la carrera que te interese, tienes diferentes directores para cada una. Por ejemplo, para la carrera de Ingeniería Mecatrónica en Latacunga, el director es Darío Mendoza C., y si necesitas contactarlo puedes escribirle al correo djmendoza@espe.edu.ec.', error=None, usage=None, raw_response='Buenos días! Puedo ayudarte con eso. La respuesta a tu pregunta sería que dependiendo la carrera que te interese, tienes diferentes directores para cada una. Por ejemplo, para la carrera de Ingeniería Mecatrónica en Latacunga, el director es Darío Mendoza C., y si necesitas contactarlo puedes escribirle al correo djmendoza@espe.edu.ec.', metadata=None)

In [25]:
class GeneratorOutput:
    def __init__(self, data=None, error=None, usage=None, raw_response=None, metadata=None):
        self.data = data
        self.error = error
        self.usage = usage
        self.raw_response = raw_response
        self.metadata = metadata

def extract_raw_response(output):
    if hasattr(output, 'raw_response') and output.raw_response is not None:
        return output.raw_response
    elif hasattr(output, 'data') and output.data is not None:
        return output.data
    return "Lo siento, no puedo procesar la respuesta en este momento."

clean_text = extract_raw_response(output)
print(clean_text)


Buenos días! Puedo ayudarte con eso. La respuesta a tu pregunta sería que dependiendo la carrera que te interese, tienes diferentes directores para cada una. Por ejemplo, para la carrera de Ingeniería Mecatrónica en Latacunga, el director es Darío Mendoza C., y si necesitas contactarlo puedes escribirle al correo djmendoza@espe.edu.ec.


In [26]:
import time
import psutil
import torch
import numpy as np

# Before calling get_embeddings, restore the original model
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Supongo que `preguntas_respuestas` y `index` están definidos y configurados.
# Si no es así, asegúrate de configurarlos antes de ejecutar este código.

# Lista de 10 prompts
prompts = [
    "voy a pasar matemáticas?",
    "¿Qué es la inteligencia artificial?",
    "Explica la teoría de la relatividad en pocas palabras.",
    "¿Cómo puedo mejorar mi productividad?",
    "Cuéntame un chiste de programación.",
    "Genera una historia corta de ciencia ficción.",
    "Dime una receta rápida y saludable.",
    "¿Cuándo es el próximo semestre en la ESPE?",
    "¿Qué servicios ofrece la ESPE?",
    "¿Cómo contacto al servicio al cliente en la ESPE?"
]

# Función para obtener el uso de memoria y CPU
def obtener_uso_memoria_cpu():
    memoria = psutil.virtual_memory().used / (1024 ** 3)  # En GB
    cpu = psutil.cpu_percent(interval=1)  # Uso de CPU en porcentaje
    return memoria, cpu

# Función para obtener el uso de GPU
def obtener_uso_gpu():
    if torch.cuda.is_available():
        memoria_gpu = torch.cuda.memory_allocated() / (1024 ** 3)  # En GB
        return memoria_gpu
    else:
        return 0  # Si no hay GPU disponible

# Lista para almacenar los resultados
resultados = []

# Ejecutar las pruebas
for prompt in prompts:
    nueva_pregunta = prompt
    nueva_embedding = get_embeddings([nueva_pregunta])[0].numpy().astype('float32')
    distances, indices = index.search(np.array([nueva_embedding]), k=3)
    similarity_percentage = 100 * (1 - distances[0][0])

    max_index = len(preguntas_respuestas) - 1
    valid_indices = [i for i in indices[0] if i <= max_index]
    respuestas_cercanas = [preguntas_respuestas[i] for i in valid_indices]

    base_datos_vectorial = "\n".join(respuestas_cercanas)

    # Obtener el uso de memoria y CPU antes de la inferencia
    memoria_antes, cpu_antes = obtener_uso_memoria_cpu()
    memoria_gpu_antes = obtener_uso_gpu()

    # Medir el tiempo de inferencia
    start_time = time.time()

    # Generar respuesta con el modelo Llama 3
    output = qa.call({"input_str": nueva_pregunta})

    end_time = time.time()

    # Obtener el uso de memoria y CPU después de la inferencia
    memoria_despues, cpu_despues = obtener_uso_memoria_cpu()
    memoria_gpu_despues = obtener_uso_gpu()

    # Calcular el tiempo de inferencia
    inference_time = end_time - start_time

    # Obtener el texto generado limpio
    clean_text = extract_raw_response(output)

    # Guardar el resultado
    resultados.append({
        "prompt": prompt,
        "generated_text": clean_text,  # Texto generado por el modelo
        "inference_time": inference_time,
        "similarity_percentage": similarity_percentage,
        "memoria_antes": memoria_antes,
        "cpu_antes": cpu_antes,
        "memoria_despues": memoria_despues,
        "cpu_despues": cpu_despues,
        "memoria_gpu_antes": memoria_gpu_antes,
        "memoria_gpu_despues": memoria_gpu_despues
    })

# Exportar resultados a un archivo de texto
with open("resultados_llama3_pruebas.txt", "w") as f:
    for res in resultados:
        f.write(f"Prompt: {res['prompt']}\n")
        f.write(f"Texto Generado: {res['generated_text']}\n")
        f.write(f"Similitud: {res['similarity_percentage']:.2f}%\n")
        f.write(f"Tiempo de Inferencia: {res['inference_time']:.2f} segundos\n")
        f.write(f"Uso de Memoria Antes: {res['memoria_antes']:.2f} GB\n")
        f.write(f"Uso de CPU Antes: {res['cpu_antes']}%\n")
        f.write(f"Uso de Memoria Después: {res['memoria_despues']:.2f} GB\n")
        f.write(f"Uso de CPU Después: {res['cpu_despues']}%\n")
        f.write(f"Uso de Memoria GPU Antes: {res['memoria_gpu_antes']:.2f} GB\n")
        f.write(f"Uso de Memoria GPU Después: {res['memoria_gpu_despues']:.2f} GB\n")
        f.write("\n" + "="*50 + "\n\n")

print("Resultados exportados a 'resultados_llama3_pruebas.txt'")

KeyboardInterrupt: 

# INTENTO API EXPOSED

In [10]:
!pip install fastapi uvicorn nest-asyncio pyngrok


In [11]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uvicorn

!ngrok config add-authtoken 2kzDHXELRRcFAebVUjUnay2qrZa_3jkkdFAEsAcsiQsM2GXup
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')


nest_asyncio.apply()

app = FastAPI()

class Query(BaseModel):
    question: str

@app.post("/ask")
async def ask_question(query: Query):
    nueva_pregunta = query.question
    nueva_embedding = get_embeddings([nueva_pregunta])[0].numpy().astype('float32')
    distances, indices = index.search(np.array([nueva_embedding]), k=3)
    similarity_percentage = 100 * (1 - distances[0][0])

    max_index = len(preguntas_respuestas) - 1
    valid_indices = [i for i in indices[0] if i <= max_index]
    respuestas_cercanas = [preguntas_respuestas[i] for i in valid_indices]

    base_datos_vectorial = "\n".join(respuestas_cercanas)

    output = qa.call({"input_str": nueva_pregunta})
    clean_text = extract_raw_response(output)

    return {"response": clean_text}

# Expose the API using ngrok
public_url = ngrok.connect(8000)
print(f"API running on {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


INFO:     Started server process [9558]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


API running on NgrokTunnel: "https://7c81-34-125-55-134.ngrok-free.app" -> "http://localhost:8000"
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1.1" 200 OK
INFO:     186.101.180.43:0 - "POST /ask HTTP/1

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [9558]
